In [ ]:
from random import randint

from evidently.legacy.ui.dashboards.reports import DashboardPanelHistogram
from evidently import BinaryClassification
from evidently.metrics import PrecisionByLabel
from evidently.metrics.column_statistics import CategoryCount
from evidently.legacy.ui.dashboards import CounterAgg
from evidently.legacy.ui.dashboards import PanelValue
from evidently.legacy.ui.dashboards import ReportFilter
import datetime
import pandas as pd

from evidently.metrics import MeanValue
from evidently.metrics import UniqueValueCount
from evidently import Dataset
from evidently import DataDefinition
from evidently import Report

from evidently.ui.backport import snapshot_v2_to_v1
from evidently.ui.backport import SingleValueDashboardPanel
from evidently.tests import lte
from evidently.legacy.ui.dashboards import DashboardPanelCounter
from evidently.legacy.ui.workspace import Workspace

def create_snapshot(i):
    df = pd.DataFrame({
        "col": list(randint(0, 5) for _ in range(i + 5)),
        "target": list(x % 2 for x in range(i + 5)),
        "prediction": list(x / 10 for x in range(i + 5)),
    })
    dataset = Dataset.from_pandas(
        df,
        data_definition=DataDefinition(
            numerical_columns=["col"],
            classification=[BinaryClassification()]
        ),
    )
    report = Report([MeanValue(column="col", tests=[lte(4)]), UniqueValueCount(column="col")])
    snapshot_v2 = report.run(dataset, None)

    snapshot_v1 = snapshot_v2_to_v1(snapshot_v2)
    import uuid6

    snapshot_v1.id = uuid6.UUID(int=i, version=7)
    snapshot_v1.timestamp = datetime.datetime.now() - datetime.timedelta(days=1)
    return snapshot_v1

ws = Workspace.create("./workspace")
from evidently.legacy.ui.type_aliases import ZERO_UUID

project = ws.get_project(ZERO_UUID)
if project is None:
    from evidently.legacy.ui.base import Project

    project = ws.add_project(Project(id=ZERO_UUID, name="test proj"))

    project.dashboard.add_panel(SingleValueDashboardPanel(metric_id=MeanValue(column="col", tests=[lte(4)]).metric_id))
    project.dashboard.add_panel(DashboardPanelCounter(
        title="Last mean value",
        agg=CounterAgg.LAST,
        value=PanelValue(field_path="value", metric_args={"metric.metric_id": "2e5caa9690281e02cf243c736d687782"}),
        filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=True),
    ))
    project.dashboard.add_panel(DashboardPanelHistogram(
        title="Distr",
        value=PanelValue(field_path="values", metric_args={"metric.type": "evidently:metric_v2:UniqueValueCount"}),
        filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=True),
    ))
    # project.dashboard.add_panel(SingleValueDashboardPanel(metric_id="mean:col"))
    project.save()

for i in range(10):
    project.add_snapshot(create_snapshot(i))    

In [ ]:
# use `evidently ui` to run UI service to see project dashboard.